In [2]:
#load file
import pandas as pd
import numpy as np
purchase_raw = pd.read_csv("./Resources/purchase_data.csv")
purchase_raw.head(5)

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


Player Count

In [3]:
#Player Count
total_player_count = purchase_raw["SN"].nunique()
total_player = pd.DataFrame({"Total Player": [total_player_count]})
total_player

,Total Player
0,576


Purchase Analysis (Total)

In [4]:
# Purchase Analysis (Total)
# Number of Unique Items
total_unique_item = purchase_raw["Item ID"].nunique()

#Total Number of Purchases
total_purchase = purchase_raw["Purchase ID"].count()

#Total Revenue
total_revenue = purchase_raw["Price"].sum()

#Average Purchase Price
avg_purchase_price = round(total_revenue/total_purchase,2)

purchase_analysis_total = pd.DataFrame({
    "Number of Unique Items": [total_unique_item],
    "Average Price": [avg_purchase_price],
    "Number of Purchases": [total_purchase],
    "Total Revenue": [total_revenue]
})
purchase_analysis_total
format_dict = {'Average Price':'${:,.2f}','Total Revenue':'${:,.2f}'}
purchase_analysis_total.style.format(format_dict)

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,183,$3.05,780,"$2,379.77"


Gender Demographics

In [5]:
#Gender Demographics
#Percentage and Count of Male Players
male_player = purchase_raw.loc[purchase_raw["Gender"]=="Male",:]
count_male_player = male_player["SN"].nunique()
percent_male_player = round(count_male_player/total_player_count,4)

#Percentage and Count of Female Players
female_player = purchase_raw.loc[purchase_raw["Gender"]=="Female",:]
count_female_player = female_player["SN"].nunique()
percent_female_player = round(count_female_player/total_player_count,4)

#Percentage and Count of Other / Non-Disclosed
count_other = total_player_count - count_male_player - count_female_player
percent_other = 1 - percent_male_player - percent_female_player

gender_demographics = pd.DataFrame({
    "": ["Male", "Female", "Other/Non-Disclosed"],
    "Total Count": [count_male_player, count_female_player, count_other],
    "Percentage of Players": [percent_male_player, percent_female_player, percent_other]
})
gender_table = gender_demographics.set_index("")
gender_format = {'Percentage of Players':'{:.2%}'}
gender_table.style.format(gender_format)

,Total Count,Percentage of Players
,,
Male,484,84.03%
Female,81,14.06%
Other/Non-Disclosed,11,1.91%


Purchase Analysis (Gender)

In [6]:
#Purchasing Analysis (Gender)
gender_list = ["Female", "Male", "Other"]
other_player = purchase_raw.loc[purchase_raw["Gender"]=="Other / Non-Disclosed",:]

#Purchase Count
male_purchase = male_player["Purchase ID"].nunique()
female_purchase = female_player["Purchase ID"].nunique()
other_purchase = total_purchase - male_purchase - female_purchase
purchase_count_list = [female_purchase,male_purchase,other_purchase]

#Average Purchase Price
avg_female_price = female_player["Price"].mean()
avg_male_price = male_player["Price"].mean()
avg_other_price = other_player["Price"].mean()
avg_price_list = [avg_female_price, avg_male_price, avg_other_price]

#Total Purchase Value
total_female_value = female_player["Price"].sum()
total_male_value = male_player["Price"].sum()
total_other_value = other_player["Price"].sum()
total_value_list = [total_female_value, total_male_value, total_other_value]

#Average Purchase Total per Person by Gender
avg_female_person = total_female_value/count_female_player
avg_male_person = total_male_value/count_male_player
avg_other_person = total_other_value/count_other
avg_total_person_list = [avg_female_person, avg_male_person, avg_other_person]

gender_purchase = pd.DataFrame({
    "Gender": ["Female", "Male","Other / Non-Disclosed"],
    "Purchase Count": purchase_count_list,
    "Average Purchase Price": avg_price_list,
    "Total Purchase Value": total_value_list,
    "Avg Total Purchase per Person": avg_total_person_list
})

gender_purchase_format = {'Average Purchase Price':'${:,.2f}', "Total Purchase Value": '${:,.2f}',"Avg Total Purchase per Person": '${:,.2f}'}
gender_purchase.set_index("Gender")
gender_purchase.style.format(gender_purchase_format)

,Gender,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
0,Female,113,$3.20,$361.94,$4.47
1,Male,652,$3.02,"$1,967.64",$4.07
2,Other / Non-Disclosed,15,$3.35,$50.19,$4.56


Age Demographics

In [7]:
#Age Demographics
bins = [0,9,14,19,24,29,34,39,100]
bins_label = ["<10","10-14","15-19","20-24","25-29","30-34","35-39","40+"]
purchase_raw["Age Group"] = pd.cut(purchase_raw["Age"], bins, labels = bins_label, include_lowest = True)
age_group_count = purchase_raw.groupby(["Age Group"]).nunique()
age_group_count = age_group_count.drop(columns = ["Purchase ID","Age","Gender","Item ID", "Item Name","Price","Age Group"]).rename(columns = {"SN":"Total Count"})
age_group_count["Percentage of Players"] = round(age_group_count["Total Count"]/age_group_count["Total Count"].sum(),4)
age_group_count["Percentage of Players"] = (age_group_count["Percentage of Players"]*100).map("{:,.2f}%".format)
age_group_count

,Total Count,Percentage of Players
Age Group,,
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


Top 5 Spenders
by total Purcase Value

In [8]:
#Top 5 Spenders by total purchase value
person_total_purchase = purchase_raw.groupby("SN").sum()
ranked_spenders = person_total_purchase.sort_values(["Price"],ascending = False)
top_5_spender = ranked_spenders.head(5)
top_5_dropped = top_5_spender.drop(columns = ["Purchase ID","Age","Item ID"])

person_number_purchase = purchase_raw.groupby("SN").count()
person_purchase_count = person_number_purchase.drop(columns = ["Age", "Gender","Item ID", "Item Name", "Price"])

top_5_spenders_format = pd.merge(top_5_dropped, person_purchase_count, on = "SN", how = "left")
top_5_spenders_format["Average Purchase Price"] = top_5_spenders_format["Price"]/top_5_spenders_format["Purchase ID"]
top_5_spenders_format = top_5_spenders_format[["Purchase ID", "Average Purchase Price","Price"]]
top_5_spenders_final = top_5_spenders_format.rename(columns = {"Price":"Total Purchase Value", "Purchase ID":"Purchase Count"})

top_5_spender_tableformat = {'Average Purchase Price':'${:,.2f}', "Total Purchase Value": '${:,.2f}'}
top_5_spenders_final.style.format(top_5_spender_tableformat)



,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


Top 5 Pupolar Items
by Purchase Count

In [9]:
# 5 Most Popular Items by purchase count
person_purchase_count = purchase_raw.groupby("Item ID").count()
ranked_purchase_count = person_purchase_count.sort_values(["Purchase ID"], ascending = False)
top_5_purchase_count = ranked_purchase_count.head(5)
top_5_purchase_count_2 = top_5_purchase_count.drop(columns = ["SN", "Age","Gender","Item Name", "Price"])

total_purchase_table = purchase_raw.groupby("Item ID").sum()
total_purchase_table_2 = total_purchase_table.drop(columns = ["Purchase ID", "Age"])

top_5_item_pre = pd.merge(top_5_purchase_count_2, total_purchase_table_2, on = "Item ID", how = "left")
top_5_item_pre_2 = top_5_item_pre.rename(columns = {"Purchase ID": "Purchase Count", "Price": "Total Purchase Value"})

purchase_raw_dropped = purchase_raw.drop(columns=["Purchase ID", "SN", "Age","Gender","Age Group"]).drop_duplicates()

top_5_item = pd.merge(top_5_item_pre_2, purchase_raw_dropped, on = "Item ID", how = "left")
top_5_item = top_5_item.rename(columns = {"Price": "Item Price"}).set_index(["Item ID", "Item Name"])
top_5_item = top_5_item[["Purchase Count", "Item Price", "Total Purchase Value"]]
top_5_item_tableformat = {'Item Price':'${:,.2f}', "Total Purchase Value": '${:,.2f}'}
top_5_item.style.format(top_5_item_tableformat)

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
82,Nirvana,9,$4.90,$44.10
19,"Pursuit, Cudgel of Necromancy",8,$1.02,$8.16


Top 5 Most Profitable Items
by Total Purchase Value

In [10]:
#5 Most Profitable Items (by total purchase value)
top_5_profitable = total_purchase_table.sort_values(["Price"],ascending = False).head(5)
top_5_profitable = top_5_profitable.drop(columns = ["Purchase ID","Age"])
item_purchase_count = person_purchase_count.drop(columns = ["SN","Age","Gender","Item Name","Price"])
most_profitable_pre = pd.merge(top_5_profitable, item_purchase_count,on = "Item ID", how = "left")
most_profitable = pd.merge(most_profitable_pre,purchase_raw_dropped, on = "Item ID", how = "left")
most_profitable = most_profitable[["Item ID", "Item Name","Purchase ID", "Price_y","Price_x"]].set_index(["Item ID", "Item Name"])
most_profitable = most_profitable.rename(columns = {"Purchase ID": "Purchase Count", "Price_y":"Item Price", "Price_x":"Total Purchase Value"})
most_profitable_format = {'Item Price':'${:,.2f}', "Total Purchase Value": '${:,.2f}'}
most_profitable.style.format(most_profitable_format)

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
92,Final Critic,8,$4.88,$39.04
103,Singed Scalpel,8,$4.35,$34.80
